# Predicting The House Prices of Banglore Using LinearRegression

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression



# Reading Data

In [ ]:
data=pd.read_csv("/Users/jaypanchal/aiml/data/Bengaluru_House_Data.csv")

In [ ]:
print(data.shape)
print(data.info())

In [ ]:
data.head(5)

In [ ]:
data.isnull().sum() # Contains many null values lets analyze and try to fill those

In [ ]:
data=data.drop(columns=['availability','society'])

In [ ]:
for col in ["location","size","bath","balcony"]:
   print(f"{col} : {data[col].value_counts()}")

In [ ]:

def fill_na(data,data_cols):
    for col in data_cols:
        if col=="location":
            data[col]=data[col].fillna('Whitefield')
        elif col=="size":
            data[col]=data[col].fillna('2 BHK')
        elif col=="bath":
            data[col]=data[col].fillna(data[col].median) 
        elif col=="balcony":
            data[col] = data[col].fillna(data[col].mode()[0])
    return data
            
data_cols=["location","size","bath","balcony"]
data=fill_na(data,data_cols)

In [ ]:
data.isnull().sum()

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
data.tail(5)

In [ ]:
for i in data.columns:
    print(data[i].unique())

In [ ]:
def convert_sqft(x):
    try:
        if '-' in x:
            tokens = x.split('-')
            return (float(tokens[0]) + float(tokens[1])) / 2
        return float(x)
    except:
        return np.nan

data['total_sqft'] = data['total_sqft'].apply(convert_sqft)
data = data.dropna(subset=['total_sqft'])

In [ ]:
data['bhk'] = data['size'].apply(lambda x: int(x.split(' ')[0]))
data = data.drop('size', axis=1)

In [ ]:
data['bath'] = pd.to_numeric(data['bath'], errors='coerce')
data = data.dropna(subset=['bath'])
data['bath'] = data['bath'].astype(int)


In [ ]:
data['balcony'] = data['balcony'].fillna(data['balcony'].median())
data['balcony'] = data['balcony'].astype(int)


In [ ]:
data['location'] = data['location'].str.strip()
location_counts = data['location'].value_counts()
data['location'] = data['location'].apply(lambda x: 'other' if location_counts[x] <= 10 else x)
data = pd.get_dummies(data, columns=['area_type', 'location'], drop_first=True)

In [ ]:
data = data[(data['total_sqft'] >= 300) & (data['total_sqft'] <= 10000)]

In [ ]:
data = data[(data['price'] >= 10) & (data['price'] <= 500)]


In [ ]:
data.head()

In [ ]:
data.describe()

In [ ]:
X = data.drop(['price'], axis=1)
y = data['price']

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:

model = LinearRegression()
model.fit(X_train, y_train)


In [ ]:

print("R2:", model.score(X_test, y_test))